# Simple Astra run

In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

In [2]:
from astra import Astra,  AstraGenerator, tools, template_dir, run_astra, run_astra_with_generator
import os

In [3]:
# Path to ASTRA executable file
MY_ASTRA_BIN = '$HOME/Code/astra/bin/Astra'

# Input template file 
MY_TEMPLATE = os.path.join(template_dir, 'dcgun/astra.in')


In [4]:
a = Astra(input_file=MY_TEMPLATE, astra_bin=MY_ASTRA_BIN)

In [5]:
# Change some inputs
a.input['newrun']['zemit']  = 1000
a.input['newrun']['zphase'] = 2
a.input['newrun']['phases'] = True
a.input['newrun']['zstop']  = 1


In [6]:
a.verbose = True
a.run()

init dir:  /Users/chrisonian/Code/GitHub/lume-astra/examples
running astra in /private/var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmpofu4he_l/astra
 --------------------------------------------------------------------------

               Astra- A space charge tracking algorithm 
                            Version 3.0              
                        DESY,  Hamburg 2011          
                        Tue Dec  3 10:32:37 

     Parameter file is:  astra.in                                          
     astra input file for L0 injector (20070501_1)                                   

 Initialize element settings:
     neglecting space charge forces 

 --------------------------------------------------------------------------
 Cavity:

     Reading cavity field data from:      /Users/chrisonian/Code/GitHub/lume-astra/templates/dcgun/dcgun_GHV.dat          
     maximum gradient                     -11.60     MV/m
     at                                   3.9750E-02 m
     e

In [7]:
a.output.keys()

dict_keys(['z', 't', 'x_average', 'x_rms', 'xp_rms', 'x_normemit', 'xxp_average', 'y_average', 'y_rms', 'yp_rms', 'y_normemit', 'yyp_average', 'E_kinetic', 'z_rms', 'deltaE_rms', 'z_normemit', 'zEp_average', 'z_for_landf', 'n_particles', 'total_charge', 'n_lost', 'energy_deposited', 'energy_exchange', 'start_time', 'run_time', 'run_error'])

In [8]:
a.input_file

'/var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmpofu4he_l/astra/astra.in'

In [9]:
a.path

'/var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmpofu4he_l/astra/'

In [10]:
a.load_screens()

loading 2 screens
[50.0, 100.0]


In [11]:
a.screen[0].keys()

dict_keys(['x', 'y', 'z_ref', 'z_rel', 'px', 'py', 'pz_ref', 'pz_rel', 't_ref', 't_rel', 'qmacro', 'status', 'species_index'])

# Alternative: run_astra

This is the functional way to run astra and return the evaluate Astra object. 

Settings is a list of settings that can appear in the input file.

In [12]:
settings0 = {'zstop':1, 'zemit':200, 'zphase':1, 'phases':True}

# Just run astra
a2 = run_astra(settings0, astra_input_file=MY_TEMPLATE, verbose=True, timeout=100)

run_astra
zstop is in astra newrun
zemit is in astra newrun
zphase is in astra newrun
phases is in astra newrun
init dir:  /Users/chrisonian/Code/GitHub/lume-astra/examples
running astra in /private/var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmpk8ngr4fs/astra


In [13]:
MY_GENERATOR_TEMPLATE = os.path.join(template_dir, 'dcgun/generator.in')

# Run Astra with Generator
settings0['ipart']= 1234

a3 = run_astra_with_generator(settings0, astra_input_file=MY_TEMPLATE,
                              generator_input_file=MY_GENERATOR_TEMPLATE, verbose=True)

run_astra_with_generator
zstop is in astra newrun
zemit is in astra newrun
zphase is in astra newrun
phases is in astra newrun
ipart is in generator
set spacecharge mesh for n_particles: 1234 to {'nrad': 9, 'nlong_in': 17}
 --------------------------------------------------------------------------

                              generator              
                             Version 1.0             
                        DESY,  Hamburg 2002          
                        Tue Dec  3 10:32:41 

     Working File is:    temp_generator.in                                 
     Initializing       1234      electrons
     including 6 probe particles at standard positions
     Particles start from a cathode
     Particles are quasi randomly distributed

          Energy spread too high.
                  28 times standard correction procedure



     Final check:
     Particles taken into account      N =       1234
     total charge                      Q =    -0.1000     nC
     ho

# Archive screens, output to HDF5 in openPMD beamphysics format

In [14]:
H5FILE='astra.h5'

In [15]:
a.archive(H5FILE)

Archiving to file astra.h5


'astra.h5'

In [16]:
# If no file is given, a filename will be invented based on the fingerprint
H5FILE2 = a2.archive()

Archiving to file astra_eb97c97d2fa573e20dcd632aefd800ee.h5


# Read openPMD

In [17]:
import h5py
h5 = h5py.File(H5FILE, 'r')

In [18]:
# Base attributes
dict(h5.attrs)

{'basePath': b'/screen/%T/',
 'dataType': b'openPMD',
 'openPMD': b'2.0.0',
 'openPMDextension': b'BeamPhysics;SpeciesType',
 'particlesPath': b'/'}

In [19]:
# Groups
list(h5)[0:10]

['input', 'output', 'screen']

In [20]:
# Screens
list(h5['screen/'])[0:10]

['0', '1']

# Plot openPMD beamphysics

In [21]:
from opmd_beamphysics import bunch_plotting, bunch_tools, bunch_stats

from bokeh.plotting import  show, output_notebook
from bokeh.layouts import column, row
output_notebook(verbose=False, hide_banner=True)

In [22]:
# Pick a screen
bunch1 = h5['screen/0']

# Plot
show(row(
    bunch_plotting.plot_bunch_h5(bunch1, 'x', 'px', bins = 50, liveOnly=False),
    bunch_plotting.plot_bunch_h5(bunch1, 'z', 'pz', bins = 50, liveOnly=False)
))
    
    

In [24]:
# Cleanup
os.remove(H5FILE)
os.remove(H5FILE2)